In [33]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Retreive class from python file
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Username and password to connect to database via CRUD Module

username = "aacuser"
password = "SNHU1234"
db = AnimalShelter(username, password)

# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Image anchor with blank target to open in new window
    html.A([html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                 height = 250, width = 251))], href = 'https://www.snhu.edu', target = "_blank"),
    # Unique Indentifier
    html.Center(html.H4('By Shoshanna Jackson')),
    html.Hr(),
    # Radio button filters
    html.Div(
        dcc.RadioItems(
        id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain Rescue', 'value': 'MOUNTAIN'},
                {'label': 'Disaster Rescue', 'value': 'DISASTER'},
                {'label': 'Reset', 'value': 'RESET'}
             ])),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        row_selectable="single", # Allows row selection
        selected_rows=[0], # Default first row selected
        filter_action="native", # Allows a filter
        sort_action="native", # Allows sorting
        page_action="native", # Enables pagination
        page_current=0, # Sets start page as first
        page_size=10 # Limits rows to 10 per page
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so pie chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 # Pie Chart
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                     style={"width":"45%"}
                 ),
                 # Geolocation Map
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                     style={"width":"45%"}
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
# Filter interactive data table with MongoDB queries
def update_dashboard(filter_type):
    if filter_type == 'WATER':
        df = pd.DataFrame.from_records(db.read( {'$and': [ 
            {'$or': [{'breed':'Labrador Retriever Mix'}, 
                     {'breed':'Chesapeake Bay Retriever'},
                     {'breed':'Newfoundland'}]}, 
            {'sex_upon_outcome':'Intact Female'}, 
            {'age_upon_outcome_in_weeks':{"$gte":26.0,"$lte":156.0}}
        ]}))        

    elif filter_type == 'MOUNTAIN':
        df = pd.DataFrame.from_records(db.read({'$and': [ 
            {'$or': [{'breed':'German Shepherd'}, 
                     {'breed':'Alaskan Malamute'}, 
                     {'breed':'Old English Sheepdog'}, 
                     {'breed':'Siberian Husky'}, 
                     {'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, 
            {'age_upon_outcome_in_weeks':{"$gte":26.0,"$lte":156.0}}
        ]}))

    elif filter_type == 'DISASTER':
        df = pd.DataFrame.from_records(db.read({'$and': [ 
            {'$or': [{'breed':'Doberman Pinscher'}, 
                     {'breed':'German Sheperd'}, 
                     {'breed':'Golden Retriever'}, 
                     {'breed':'Bloodhound'}, 
                     {'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, 
            {'age_upon_outcome_in_weeks':{"$gte":20.0,"$lte":300.0}}
        ]}))

    elif filter_type == 'RESET':
        df = pd.DataFrame.from_records(db.read({}))
   
    # Return cleaned table data
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

# This callback displays the quantity of breeds of animal 
# based on filter selection in a pie chart
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]

    
# This callback will highlight a cell on the data table when the user selects it
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"),
               Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Center map based on selection 
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row,13],dff.iloc[row,14]], zoom=9, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H2("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:10245/
